# Driving and Reinforcement Learning

Last time we reduce the driving problem to mimicing a human driver (behavior cloning). This then became the supervised learning problem of predicting actions from observations. While we saw that this performed at a decent level and was able to drive around the track, we also noted some important drawbacks of this approach.

Today, we'll look at a different approach to solving the same problem - reinforcement learning. In partcular, we'll see how we can teach a driver how to drive without explicitly giving it any human guidance! The AI will learn through repeated trial and error!

Let's get started!

In [12]:
# %load_ext autoreload
# %autoreload 2
# %load_ext tensorboard

INDEX2STRING = [
    "stay",
    "accelerate",
    "right",
    "left",
]

import gym 
from common.utils import *
from common.driving_utils import *
import plotly.express as px
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import random
from collections import Counter, defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, Conv2D, MaxPool2D, BatchNormalization, concatenate, Lambda, Flatten
from tensorflow.keras.optimizers import Adam
import os
import datetime
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
from bc.blackjack import run_blackjack, human_blackjack_player
from tqdm import tqdm

def run_racing(policy):
    step = 0
    env = gym.make("CarRacing-v0")
    obs = env.reset() # Intialize the observation
    while True:
        env.render()
        move = policy(obs, env, step)
        obs, reward, done, info = env.step(move)
        if done:
            break
        step += 1
    env.close()

## Q-Learning

We'll use a famous RL algorithm called Q-Learning. Before we apply Q-Learning to the complex problem of driving, let's try to get a handle on the algorithm itself! Here we'll use state and observation interchangeably.

Recall we have the following setup: 

![setup](https://www.novatec-gmbh.de/wp-content/uploads/1_mPGk9WTNNvp3i4-9JFgD3w.png)

### What is Q?
Q is a function that determines the **quality** of a certain (state, action) pair. Or and estimate of the best possible total reward I get for taking an action at a certain state. 

**Question:** If I have such a function, what policy should I use at a certain state?

TODO Make this hidden

If I'm at state $s$, I can use $Q$ to check which action $a$ has the highest quality, and then choose that action to perform!

Mathematically, I want to pick the action $a$ that maximizes $Q(s, a)$.

**Exercise:** Write this policy! Let's imagine that there are only two possible actions 0 and 1. 
* Access the $Q(s, a)$ using $q[(s, a)]$

In [2]:
def q_policy(s, q):
    return int(q[(s, 0)] < q[(s, 1)])
    # return 0 if q[(obs, 0)] >= q[(obs, 1)] else 1



Awesome! The tricky part is then learning the $Q$ function itself (hence the name $Q$-learning) The idea is just to run a bunch of random simulations and depending on the reward we get at each step update our quality values for certain (state, action) pairs. In particular, we update the q-values using the following equation:

$$Q_{new}(s_t, a_t) = (1-\alpha)Q_{old}(s_t,a_t) + \alpha[r_t + \gamma \max_a(Q_{old}(s_{t+1}, a))]$$

That's a big equation so let's break it down:
* $Q_{new}(s_t, a_t)$ is our updated estimate for the quality of a certain (state, action) pair
* $Q_{old}(s_t,a_t)$ is our old estimate for the 
* $r_t + \gamma \max_a(Q_{old}(s_{t+1}, a))$. This gives a new estimate of $Q(s, a)$ based on a simulation and our prior estimate. It is calculated as  the reward that we get for taking $a_t$ at state $s_t$ plus the best possible total reward I can get at the next state using our old estimates. $\gamma$ is known as a discount factor which dictates how much we care about rewards in the distant future as compared to rewards now.
* $\alpha$ is our learning rate which dictates how much we want to update our beliefs on the for each simulation.

Rephrased: The above equation updates our current estimate of $Q(s, a)$ by shifting it towards our new estimate from simulations by a factor of $\alpha$.

Don't worry if that all seems confusing we'll implement Q-learning ourselves from scratch!

## Blackjack

To get the hang of Q-learning, let's use it to play blackjack! Here are some quick rules:
* You start of with two cards
* You can 'hit' which is to draw another card. Or 'stay' which is to say you don't want more cards
* The goal is to have your cards add up to larger than the dealers cards but less than 21.
* You can see one of the dealers cards
* An ace can count as either 1 or 11 depending on which gives you the higher score

The **observations** have the following form (my total, dealer card, usable ace)

The **actions** are either 'hit' or 'stay'

The **reward** is 1 for a win, 0 for a tie and -1 for a draw.

Let's play a couple of rounds!

In [3]:
history = run_blackjack(human_blackjack_player, show=True)

Your total: 14
Dealer card: 8
Usable ace?: False
0 for stay 1 for hit: 


ValueError: invalid literal for int() with base 10: ''

Cool! Now that you have some intuition for the game, let's look at what data we record for each simulation.

In [4]:
history[0]

The elements of the tuple are the following 
1. $s_t$
2. $a_t$ (hit or stay)
3. $s_{t+1}$ the state we end up in after taking the action
4. True if the episode is over, False otherwize.
5. $r_t$ reward at this time step

Given this information, we can then use our equation to update our estimates for $Q$! As it is tiring to play many games of blackjack, we instead let our q-learning agent play by itself against the dealer, with the hopes that it learns from its experience!

## Exploration vs Exploitation
When the q-learning agent plays what policy should it use in order to get the most experience? 
**Question:** Think back to when you first stated to play a certain game, what did you do in order to get better?

Maybe it was a mix of trying new strategies and using the ones that worked and fine tuning them! That's exactly what we'll do here! We'll use a policy called the $\epsilon$-greedy policy, which tries new (random strategies) with probability $\epsilon$ and uses strategies that it knows will work the rest of the time!

**Exercise:** Write the epsilon greedy policy: 
* `eps` is the probability we select a random action
* `random.random()` gives us a random number between 0 and 1
* How can we use the above to do one thing with probabiliy $\epsilon$?
* `random.choice([0,1])` gives us a random action
* remember we wrote a function `q_policy(s, q)` above which gives us the best action to take given our estimate for $Q$.

In [14]:
def eps_policy(s, q, eps):
    explore = random.random() <= eps
    if explore:
        return random.choice([0, 1])
    else:
        return q_policy(s, q)

In [17]:
def q_learn_blackjack(eps, gamma, alpha, epochs):
    q_table = defaultdict(int)
    training_results = []
    for _ in tqdm(range(epochs)):
        history = run_blackjack(lambda obs: eps_policy(obs, q_table, eps))
        for s, a, s_next, done, r in reversed(history):
            old = q_table[(s, a)]
            if done:
                q_table[(s, a)] = (1 - alpha) * old + alpha * (r)
            else:
                best_next = max([q_table[(s_next, 0)], q_table[(s_next, 1)]])
                q_table[(s, a)] = (1 - alpha) * old + alpha * (r + gamma * best_next)
        training_results.append(history[-1][-1])
    return q_table, training_results

In [87]:
EPS = 0.7
GAMMA = 1
ALPHA = 0.1
EPOCHS = 200000
q_table, results = q_learn_blackjack(EPS, GAMMA, ALPHA, EPOCHS)

100%|██████████| 200000/200000 [10:39<00:00, 312.85it/s]


In [47]:
q_table

defaultdict(int,
            {((19, 9, False), 0): 0.31466792587523784,
             ((19, 9, False), 1): -0.6766881888584568,
             ((19, 10, False), 1): -0.7790884883309752,
             ((20, 5, False), 0): 0.6878451707986347,
             ((20, 5, False), 1): -0.8106965907465763,
             ((18, 10, True), 0): -0.20725806860273033,
             ((15, 1, False), 0): -0.7295446640666589,
             ((15, 1, False), 1): -0.5862415384088506,
             ((17, 10, False), 1): -0.5765289556980614,
             ((9, 10, False), 1): -0.20783509004338688,
             ((17, 10, False), 0): -0.51296163127592,
             ((19, 10, False), 0): 0.037251182408458684,
             ((12, 9, False), 0): -0.5521229071417244,
             ((19, 1, False), 1): -0.7087810272017296,
             ((4, 10, False), 0): -0.26136350939369796,
             ((4, 10, False), 1): -0.23236068258671042,
             ((16, 4, False), 0): -0.14416634858932695,
             ((16, 4, False), 1): -0.4506

In [85]:
run_blackjack(lambda s : q_policy(s, q_table), show=1)

Tie!
Your cards:  [10, 10]
Dealers cards [10, 10]


[((20, 10, False), 0, (20, 10, False), True, 0.0)]